# ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [21]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [22]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/dhpaulino/projects/udacity/data_engineer_fun/data_modeling_cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [23]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [24]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Ingest, model and query on Apache Cassandra

##  Ingested file: <font color=red>event_datafile_new.csv</font>. The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [25]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [26]:
query = '''CREATE KEYSPACE IF NOT EXISTS sparkify
WITH REPLICATION = 
{
    'class' : 'SimpleStrategy',
    'replication_factor' : 1
}'''
session.execute(query)

#### Set Keyspace

In [27]:
session.set_keyspace("sparkify")

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Thinking about query 1
Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [28]:
query = ''' CREATE TABLE IF NOT EXISTS session_items
(
    sessionid int,
    itemInSession int,
    artist text,
    song text,
    song_length float,
    PRIMARY KEY ((sessionid, itemInSession))
)'''
session.execute(query)

                    

### Motive of the modeling
Chosen to use `sessionid` and `itemInSession` to partition the data since they are the filters applied to search

### Inserting data in the table

In [29]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = '''INSERT INTO session_items (sessionid, itemInSession, artist, song, song_length)'''
        query = query + " VALUES(%s, %s, %s, %s, %s)"
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[-3]), int(line[3]), line[0], line[-2], float(line[5])))

### Running the query

In [30]:
query = '''SELECT artist, song, song_length FROM session_items WHERE sessionId = 338 AND itemInSession = 4'''
result = session.execute(query)
for row in result:
    print(row)


Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


## Thinking about query 2
Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [31]:
query = ''' CREATE TABLE IF NOT EXISTS user_session_by_item
(
    userid int,    
    sessionid int,
    itemInSession int,
    artist text,
    song text,
    firstName text,
    lastName text,
    PRIMARY KEY ((userid, sessionid), itemInSession)
)'''
session.execute(query)

                    

### Motive of the modeling
Chosen to use `userid` and `sessionid` to partition the data since they are the filters applied to search. Also made `itemInSession` a clustering key, since we want the result ordened by it and that exactly what a clustering key does with the partition data

### Inserting data in the table

In [32]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = '''INSERT INTO user_session_by_item (userid, sessionid, itemInSession, artist, song, firstName, lastName)'''
        query = query + " VALUES(%s, %s, %s, %s, %s, %s, %s)"
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[-1]), int(line[-3]), int(line[3]), line[0], line[-2], line[1], line[4]))

### Running the query

In [33]:
query = '''SELECT itemInSession, artist, song, firstName, lastName FROM user_session_by_item WHERE userId = 10 AND sessionId = 182'''
result = session.execute(query)
for row in result:
    print(row)


Row(iteminsession=0, artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(iteminsession=1, artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(iteminsession=2, artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(iteminsession=3, artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


## Thinking about query 3
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [34]:
query = ''' CREATE TABLE IF NOT EXISTS songs_listen
(  
    song text,
    userid int,
    firstName text,
    lastName text,
    PRIMARY KEY ((song), userid)
)'''
# partitioning by the condition
# used on the primary key to make the rows unique, but made clustering key to not mess with data distriibution
session.execute(query)

                    

### Motive of the modeling
Chosen to use `song` to partition the data since itis the filter applied to search. Also, used `userid` on the primary key to make the rows unique, but made part of the clustering key istead of the partition key, so it doesn't mess with data distriibution

### Inserting data in the table

In [35]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = '''INSERT INTO songs_listen (song, userid, firstName, lastName)'''
        query = query + " VALUES(%s, %s, %s, %s)"
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[-2], int(line[-1]), line[1], line[4]))

### Running the query

In [36]:
query = '''SELECT firstName, lastName FROM songs_listen WHERE song = 'All Hands Against His Own' '''
result = session.execute(query)
for row in result:
    print(row)


Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [37]:
session.execute("DROP TABLE session_items")
session.execute("DROP TABLE user_session_by_item")
session.execute("DROP TABLE songs_listen")

### Close the session and cluster connection¶

In [38]:
session.shutdown()
cluster.shutdown()